<a href="https://colab.research.google.com/github/awannabecs/PublicTensorflowRepository/blob/main/CHATBOT**.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
### import the will-be-needed modules ####
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random

In [6]:
### load the data ####
data_path =  "/content/DATAFILE/human_text.txt"
data_path2 = "/content/DATAFILE/robot_text.txt"

In [7]:
with open(data_path, mode="r",encoding="utf-8") as f :
  lines = f.read().split("\n")

with open(data_path2, mode="r", encoding="utf-8") as f :
  lines2 = f.read().split("\n")

In [8]:
lines = [re.sub(r"\[\w+\]",'hi',line) for line in lines]
lines = [" ".join(re.findall(r"\w+",line)) for line in lines]

lines2 = [re.sub(r"\[\w+\]",'',line) for line in lines2]
lines2 = [" ".join(re.findall(r"\w+",line)) for line in lines2]

pairs = list(zip(lines,lines2))

random.shuffle(pairs)

In [9]:
input_docs = []
target_docs = []
input_tokens = set()
target_tokens = set()

for line in pairs:
  input_doc, target_doc = line[0],line[1]
  input_docs.append(input_doc)
  target_doc = " ".join(re.findall(r"[\w']+|[^\s\w]", target_doc))
  target_doc = '<START> ' + target_doc + ' <END>'
  target_docs.append(target_doc)

  for token in re.findall(r"[\w']+|[^\s\w]", input_doc):
    if token not in input_tokens:
      input_tokens.add(token)
  for token in target_doc.split():
    if token not in target_tokens:
      target_tokens.add(token)

input_tokens = sorted(list(input_tokens))
target_tokens = sorted(list(target_tokens))
num_encoder_tokens = len(input_tokens)
num_decoder_tokens = len(target_tokens)

In [10]:
input_features_dict = dict(
    [(token, i) for i, token in enumerate(input_tokens)])
target_features_dict = dict(
    [(token, i) for i, token in enumerate(target_tokens)])

reverse_input_features_dict = dict(
    (i, token) for token, i in input_features_dict.items())
reverse_target_features_dict = dict(
    (i, token) for token, i in target_features_dict.items())

In [22]:
# Maximum length of sentences in input and target documents
max_encoder_seq_length = max([len(re.findall(r"[\w']+|[^\s\w]", input_doc)) for input_doc in input_docs])
max_decoder_seq_length = max([len(re.findall(r"[\w']+|[^\s\w]", target_doc)) for target_doc in target_docs])
encoder_input_data = np.zeros(
    (len(input_docs), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
encoder_input_data
decoder_input_data = np.zeros(
    (len(input_docs), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_docs), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
for line, (input_doc, target_doc) in enumerate(zip(input_docs, target_docs)):
    for timestep, token in enumerate(re.findall(r"[\w']+|[^\s\w]", input_doc)):
        # Assign 1. for the current line, timestep, & word in encoder_input_data
        encoder_input_data[line, timestep, input_features_dict[token]] = 1.
    
    for timestep, token in enumerate(target_doc.split()):
        decoder_input_data[line, timestep, target_features_dict[token]] = 1.
        if timestep > 0:
            decoder_target_data[line, timestep - 1, target_features_dict[token]] = 1.

In [30]:
from tensorflow import keras
from keras.layers import Input, LSTM, Dense
from keras.models import Model
#Dimensionality
dimensionality = 256
#The batch size and number of epochs
batch_size = 10
epochs = 50
#Encoder
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder_lstm = LSTM(dimensionality, return_state=True)
encoder_outputs, state_hidden, state_cell = encoder_lstm(encoder_inputs)
encoder_states = [state_hidden, state_cell]
#Decoder
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(dimensionality, return_sequences=True, return_state=True)
decoder_outputs, decoder_state_hidden, decoder_state_cell = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [31]:
#Model
training_model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
#Compiling
training_model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'], sample_weight_mode='temporal')
#Training
training_model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size = batch_size, epochs = epochs, validation_split = 0.2)
training_model.save('training_model.h5')

Epoch 1/600
32/32 [==============================] - 69s 2s/step - loss: 0.4783 - accuracy: 0.0183 - val_loss: 0.4923 - val_accuracy: 0.0093
Epoch 2/600
32/32 [==============================] - 60s 2s/step - loss: 0.4614 - accuracy: 0.0092 - val_loss: 0.4904 - val_accuracy: 0.0093
Epoch 3/600
32/32 [==============================] - 61s 2s/step - loss: 0.4581 - accuracy: 0.0092 - val_loss: 0.4882 - val_accuracy: 0.0092
Epoch 4/600
32/32 [==============================] - 59s 2s/step - loss: 0.4557 - accuracy: 0.0091 - val_loss: 0.4880 - val_accuracy: 0.0093
Epoch 5/600
32/32 [==============================] - 63s 2s/step - loss: 0.4537 - accuracy: 0.0092 - val_loss: 0.4891 - val_accuracy: 0.0093
Epoch 6/600
32/32 [==============================] - 60s 2s/step - loss: 0.4524 - accuracy: 0.0092 - val_loss: 0.4883 - val_accuracy: 0.0093
Epoch 7/600
32/32 [==============================] - 63s 2s/step - loss: 0.4508 - accuracy: 0.0085 - val_loss: 0.4895 - val_accuracy: 0.0093
Epoch 8/600
3

KeyboardInterrupt: ignored